In [ ]:
import pandas as pd
import networkx as nx
import pickle
import json
from itertools import combinations
from tqdm import tqdm
import re
import collections
import matplotlib.pyplot as plt
from collections import defaultdict
import seaborn as sns

## Seed for LLM

In [ ]:

with open("indicator_graph_edges_weighted.pkl", "rb") as f:
    DG = pickle.load(f)

In [ ]:
stage_skill = {}
# Iterate over each node to count outgoing connections by last stage label
for node in DG.nodes():
    node_stage = DG.nodes[node].get('stage', None)
    node_skill = DG.nodes[node].get('skill', None)

    if node_stage not in stage_skill:
        stage_skill[node_stage] = []

    if node_skill not in stage_skill[node_stage]:
        stage_skill[node_stage].append(node_skill)

print("Stage and Skills:")
for stage, skills in stage_skill.items():
    print(f"Stage {stage}: {', '.join(skills)}")

In [ ]:
text_type_seed = []

for node in DG.nodes():
    node_stage = DG.nodes[node].get('stage', None)
    node_skill = DG.nodes[node].get('skill', None)
    node_subskill = DG.nodes[node].get('subskill', None)
    node_goal = DG.nodes[node].get('goal', None)
    node_age_group = DG.nodes[node].get('age_group', None)
    node_indicator = DG.nodes[node].get('label', None)
    node_id = node

    if node_stage is not None and node_skill is not None and node_subskill is not None and node_goal is not None and node_age_group is not None and node_indicator is not None:
        text_type_seed.append({
            "indicator": node_indicator,
            "age_group": node_age_group,
            "skill": node_skill,
            "subskill": node_subskill,
            "goal": node_goal,
            "stage": node_stage,
            "id" : node_id
        })

print(len(DG.nodes())), print(len(text_type_seed))

In [ ]:
with open("/datadrive/pavan/az_storage/data_unorganized/skill_graph/version2/text_types_seed.jsonl", "w") as f:
    json.dump(text_type_seed, f, indent=4)

## Prompt for LLM

In [ ]:
instruction_response_prompt = {
  "system": "You are an expert in child development, skill acquisition, curriculum design, and language model pretraining. Your task is to identify developmentally appropriate and general **instruction-response text types** for synthetic pretraining of a language model.\n\nEach input includes:\n- indicator: a natural language description of the learning objective or task\n- age_group: developmental age (e.g., 0–5, 5–11, 11–14)\n- skill: broad academic or developmental domain (e.g., Mathematics, English, Scientific Reasoning)\n- subskill: a specific subdomain or area of focus (e.g., Listening, Measurement, Problem-solving)\n- goal: the purpose or nature of the learning (e.g., Application, Reflection, Evaluation)\n- stage: the curriculum stage (0 to 9, loosely corresponding to increasing age and complexity)\n\nInstructions:\nReturn a list of **general instruction-response style text types** that:\n- Are suitable for the learner's developmental stage\n- Can be used in instruction tuning and task-based language modeling\n- Involve a clearly defined instruction format that can be applied across many topics\n- Are defined at a high level of abstraction (e.g., \"explain why X occurs\", \"compare and contrast X and Y\")\n\n**CRITICALLY IMPORTANT**:\n- Provide abstract instruction formats, NOT specific prompts or questions\n- Text types should be 2-5 words describing a general instruction format\n- Each text type should be usable with ANY topic relevant to the age/skill combination\n\n**Examples of appropriate instruction-response text types**:\n- \"Compare and contrast analysis\"\n- \"Explain why reasoning\"\n- \"Step-by-step instruction\"\n- \"Open-ended reflection prompt\"\n\n**Examples of inappropriate text types** (too specific):\n- \"Explain why plants need water\"\n- \"Compare dogs and cats\"\n- \"Describe your favorite toy\"\n\nOutput Format:\nReturn your result as a JSON object with the following structure:\n\n```json\n{{\n  \"text_types\": [\"...\", \"...\", \"...\"]\n}}\n```\n\nEnsure the list is:\n- 15–20 items long\n- Abstract enough to work across many topics\n- Varied across explanation, reasoning, reflection, comparison, instruction, imagination\n- Appropriate in complexity for the given age group and learning goal\n\nOnly output the JSON object.",
  "user": "Given the following information about a learning objective, return a list of general, reusable instruction-response text formats that can serve as templates for synthetic training data:\n\n- indicator: {indicator}\n- age_group: {age_group}\n- skill: {skill}\n- subskill: {subskill}\n- goal: {goal}\n- stage: {stage}\n\nIMPORTANT: Provide ABSTRACT INSTRUCTION FORMATS (2-5 words each), not specific questions or prompts.\n\nExamples of good instruction formats:\n- \"Compare and contrast analysis\"\n- \"Explain why reasoning\"\n- \"Problem-solving walkthrough\"\n- \"Open-ended reflection prompt\"\n\nExamples of unsuitable formats (too specific):\n- \"Explain why plants need water\"\n- \"Compare dogs and cats\"\n- \"Solve this math problem\"\n\nEnsure your list contains:\n- 15 to 20 developmentally appropriate instruction formats\n- General templates that can be combined with ANY relevant topic\n- Varied instruction types that address different cognitive processes\n\nReturn only a JSON object in the following format:\n\n```json\n{{\n  \"text_types\": [\"...\", \"...\", \"...\"]\n}}\n```"
}
print(instruction_response_prompt['system'])
print("______________________")
print(instruction_response_prompt['user'])

In [ ]:
context_only_prompt = {
  "system": "You are an expert in child development, skill acquisition, curriculum design, and language model pretraining. Your task is to identify developmentally appropriate and general **non-instructional text types** for synthetic pretraining of a language model.\n\nEach input includes:\n- indicator: a natural language description of the learning objective or task\n- age_group: developmental age (e.g., 0–5, 5–11, 11–14)\n- skill: broad academic or developmental domain (e.g., Mathematics, English, Scientific Reasoning)\n- subskill: a specific subdomain or area of focus (e.g., Listening, Measurement, Problem-solving)\n- goal: the purpose or nature of the learning (e.g., Application, Reflection, Evaluation)\n- stage: the curriculum stage (0 to 9, loosely corresponding to increasing age and complexity)\n\nInstructions:\nReturn a list of **general non-instructional text types** that:\n- Are suitable for the learner's developmental stage\n- Reflect naturalistic or structured formats that don't rely on explicit instruction–response pairs\n- Can be used as abstract templates to generate content across many topics\n- Are defined at a high level of abstraction (e.g., \"peer dialogue\", \"narrative description\", \"cause-effect explanation\")\n\n**CRITICALLY IMPORTANT**:\n- Provide format categories, NOT specific content or scenarios\n- Text types should be 2-5 words that describe a general format, not complete sentences\n- Each text type should be usable with ANY topic relevant to the age/skill combination\n\n**Examples of appropriate non-instructional text types**:\n- \"Narrative story with characters\"\n- \"Peer conversation transcript\"\n- \"Process description passage\"\n- \"Personal reflection monologue\"\n\n**Examples of inappropriate text types** (too specific):\n- \"Story about a child going to the zoo\"\n- \"Conversation between friends about toys\"\n- \"Description of a butterfly's life cycle\"\n\nOutput Format:\nReturn your result as a JSON object with the following structure:\n\n```json\n{{\n  \"text_types\": [\"...\", \"...\", \"...\"]\n}}\n```\n\nEnsure the list is:\n- 15–20 items long\n- Abstract enough to work across many topics\n- Varied across narration, description, interaction, emotion, reasoning\n- Appropriate in complexity for the given age group and learning goal\n\nOnly output the JSON object.",
  "user": "Given the following information about a learning objective, return a list of general, reusable non-instructional text formats that can serve as templates for synthetic training data:\n\n- indicator: {indicator}\n- age_group: {age_group}\n- skill: {skill}\n- subskill: {subskill}\n- goal: {goal}\n- stage: {stage}\n\nIMPORTANT: Provide ABSTRACT FORMAT CATEGORIES (2-5 words each), not specific content or scenarios.\n\nExamples of good non-instructional formats:\n- \"Peer dialogue transcript\"\n- \"Sequential process description\"\n- \"Character-driven narrative\"\n- \"Emotional experience monologue\"\n\nExamples of unsuitable formats (too specific):\n- \"Conversation between friends about toys\"\n- \"Description of a butterfly's life cycle\"\n- \"Story about going to the beach\"\n\nEnsure your list contains:\n- 15 to 20 developmentally appropriate text formats\n- General templates that can be combined with ANY relevant topic\n- Varied format types that don't rely on explicit instruction-response pairs\n\nReturn only a JSON object in the following format:\n\n```json\n{{\n  \"text_types\": [\"...\", \"...\", \"...\"]\n}}\n```"
}
print(context_only_prompt['system'])
print("______________________")
print(context_only_prompt['user'])

In [ ]:
with open("./text_type_prompt4.1.json", "w") as f:
    json.dump(instruction_response_prompt, f, indent=4)

with open("./text_type_prompt4.2.json", "w") as f:
    json.dump(context_only_prompt, f, indent=4)

## Verification and correction of generations

In [ ]:
ver_ins_text_prompt = {"system": "You are an expert in child development, curriculum design, skill acquisition, and language model instruction tuning. Your task is to evaluate whether a proposed **instruction-response text type** is appropriate for use in synthetic pretraining and instruction tuning for a given learning task and learner profile.\n\nEach evaluation includes:\n- text_type: the proposed abstract instruction-response format (e.g., \"Explain why reasoning\")\n- indicator: a natural language description of the learning objective or task\n- age_group: developmental age range (e.g., 5–11)\n- skill: broad academic or developmental domain (e.g., Mathematics)\n- subskill: specific area of focus (e.g., Measurement)\n- goal: purpose or nature of learning (e.g., Application, Reflection)\n- stage: curriculum stage (0–9, with 0 being youngest learners)\n\nYour job is to decide:\n- Is this text_type a good abstract template for this task and learner profile?\n\nOutput your judgment in the following JSON format:\n\n```json\n{{\n  \"is_valid\": \"YES\" or \"NO\",\n  \"justification\": \"A concise explanation (1-2 sentences) of why this text type is or isn't appropriate for this indicator and learner profile.\"\n}}\n```\n\nEvaluation Criteria:\n- The text_type should be **abstract and generalizable**, not topic-specific or overly concrete.\n- It should be **developmentally appropriate** for the given age_group and stage.\n- It should align with the **learning goal** (e.g., reflection, analysis, reasoning, application).\n- It should be usable across a wide range of topics for the given skill/subskill.\n- It should be suitable for **instruction tuning** or **task-based language modeling** in a general-purpose educational language model.\n\nExamples of valid text_types: \"Explain why reasoning\", \"Compare and contrast analysis\", \"Open-ended reflection prompt\", \"Step-by-step instruction\"\nExamples of invalid text_types: \"Describe your favorite animal\", \"Explain the water cycle\", \"Name three triangle types\"",
                       "user": "Evaluate if this text type is appropriate for instruction tuning:\n\ntext_type: \"{text_type}\"\nindicator: \"{indicator}\"\nage_group: \"{age_group}\"\nskill: \"{skill}\"\nsubskill: \"{subskill}\"\ngoal: \"{goal}\"\nstage: {stage}\n\nReturn only a JSON object:\n\n```json\n{{\n  \"is_valid\": \"YES\" or \"NO\",\n  \"justification\": \"A brief explanation of why.\"\n}}\n```"}

ver_nins_text_prompt = {"system": "You are an expert in child development, curriculum design, skill acquisition, and language model instruction tuning. Your task is to evaluate whether a proposed **non-instructional text type** is appropriate for use in synthetic pretraining of a language model for a given learning task and learner profile.\n\nEach evaluation includes:\n- text_type: the proposed abstract non-instructional format (e.g., \"Narrative story with characters\")\n- indicator: a natural language description of the learning objective or task\n- age_group: developmental age range (e.g., 5–11)\n- skill: broad academic or developmental domain (e.g., Mathematics)\n- subskill: specific area of focus (e.g., Measurement)\n- goal: purpose or nature of learning (e.g., Application, Reflection)\n- stage: curriculum stage (0–9, with 0 being youngest learners)\n\nYour job is to decide:\n- Is this text_type a good abstract template for this task and learner profile?\n\nOutput your judgment in the following JSON format:\n\n```json\n{{\n  \"is_valid\": \"YES\" or \"NO\",\n  \"justification\": \"A concise explanation (1-2 sentences) of why this text type is or isn't appropriate for this indicator and learner profile.\"\n}}\n```\n\nEvaluation Criteria:\n- The text_type should be **abstract and generalizable**, not topic-specific or overly concrete.\n- It should be **developmentally appropriate** for the given age_group and stage.\n- It should align with the **learning goal** (e.g., reflection, analysis, reasoning, application).\n- It should be usable across a wide range of topics for the given skill/subskill.\n- It should reflect naturalistic or structured formats that don't rely on explicit instruction-response pairs.\n- It should be suitable for **synthetic pretraining** of a general-purpose educational language model.\n\nExamples of valid text_types: \"Narrative story with characters\", \"Peer conversation transcript\", \"Process description passage\", \"Personal reflection monologue\"\nExamples of invalid text_types: \"Story about a child going to the zoo\", \"Conversation between friends about toys\", \"Description of a butterfly's life cycle\"",
                        "user": "Evaluate if this text type is appropriate for synthetic pretraining:\n\ntext_type: \"{text_type}\"\nindicator: \"{indicator}\"\nage_group: \"{age_group}\"\nskill: \"{skill}\"\nsubskill: \"{subskill}\"\ngoal: \"{goal}\"\nstage: {stage}\n\nReturn only a JSON object:\n\n```json\n{{\n  \"is_valid\": \"YES\" or \"NO\",\n  \"justification\": \"A brief explanation of why.\"\n}}\n```"}

#save the prompts
with open("./ver_ins_text_prompt.json", "w") as f:
    json.dump(ver_ins_text_prompt, f, indent=4)

with open("./ver_nins_text_prompt.json", "w") as f:
    json.dump(ver_nins_text_prompt, f, indent=4)

### Generations

In [ ]:
with open("./ver_ins_text_raw.jsonl", "r") as f:
    data = json.load(f)
    
all_dicts = [o for o in data if isinstance(o['output'], dict) and o['output']['is_valid'] in ["YES", "NO"]]
non_dicts = [o for o in data if not isinstance(o['output'], dict) or o['output']['is_valid'] not in ["YES", "NO"]]
print(len(all_dicts)), print(len(non_dicts))

non_dicts[0]['output'] = {"is_valid": "NO",
 "justification": "While prompting for continuation (\"What happened next?\") can be useful, this text type is too open-ended and doesn\'t directly address the sensitive and complex task of explaining the difference between a slave and a free person, especially for this age group. It lacks the necessary framing for a nuanced and historically accurate response."}
ins_dicts = all_dicts + non_dicts

answers = {}
for i in ins_dicts:
    answers[i['output']['is_valid']] = answers.get(i['output']['is_valid'],0)
    answers[i['output']['is_valid']] += 1
print(answers)

ins_dict = {}
for i in ins_dicts:
    ins_dict[i['stage']] = ins_dict.get(i['stage'],{})
    ins_dict[i['stage']][i['skill']] = ins_dict[i['stage']].get(i['skill'], {})
    ins_dict[i['stage']][i['skill']][i['output']['is_valid']] = ins_dict[i['stage']][i['skill']].get(i['output']['is_valid'], 0)
    ins_dict[i['stage']][i['skill']][i['output']['is_valid']] +=1

In [ ]:
with open("./ver_nins_text_raw.jsonl", "r") as f:
    data = json.load(f)

all_dicts = [o for o in data if isinstance(o['output'], dict) and o['output']['is_valid'] in ["YES", "NO"]]
non_dicts = [o for o in data if not isinstance(o['output'], dict) or o['output']['is_valid'] not in ["YES", "NO"]]
print(len(all_dicts)), print(len(non_dicts))

non_dicts[0]['output'] = {"is_valid": "YES",
 "justification": "Recipe instructions naturally contain imperative verbs and can be easily adapted to include direct speech (e.g., \'Mom said, \"Add the flour now!\"\'), providing a context for experimenting with verb forms appropriate for this age group and writing goal."}
nins_dicts = non_dicts + all_dicts

answers = {}
for i in nins_dicts:
    answers[i['output']['is_valid']] = answers.get(i['output']['is_valid'],0)
    answers[i['output']['is_valid']] += 1
print(answers)

nins_dict = {}
for i in nins_dicts:
    nins_dict[i['stage']] = nins_dict.get(i['stage'],{})
    nins_dict[i['stage']][i['skill']] = nins_dict[i['stage']].get(i['skill'], {})
    nins_dict[i['stage']][i['skill']][i['output']['is_valid']] = nins_dict[i['stage']][i['skill']].get(i['output']['is_valid'], 0)
    nins_dict[i['stage']][i['skill']][i['output']['is_valid']] +=1

ins_data = {}
for i in ins_dicts:
    if i['output']['is_valid'] == 'YES':
        ins_data[i['id']] = ins_data.get(i['id'], [])
        ins_data[i['id']].append(i['text_type'])

nins_data = {}
for i in nins_dicts:
    if i['output']['is_valid'] == 'YES':
        nins_data[i['id']] = nins_data.get(i['id'], [])
        nins_data[i['id']].append(i['text_type'])

ins_d = {}
for i in ins_dicts:
    ins_d[i['id']] = ins_d.get(i['id'], {})
    ins_d[i['id']][i['output']['is_valid']] = ins_d[i['id']].get(i['output']['is_valid'], 0)
    ins_d[i['id']][i['output']['is_valid']] += 1
nins_d = {}
for i in nins_dicts:
    nins_d[i['id']] = nins_d.get(i['id'], {})
    nins_d[i['id']][i['output']['is_valid']] = nins_d[i['id']].get(i['output']['is_valid'], 0)
    nins_d[i['id']][i['output']['is_valid']] += 1

In [ ]:
bad_ins_samples = {"zero": [], "less_10": [], "less_15": []}
good_ins_samples = []
for j in ins_d:
    if 'YES' not in ins_d[j].keys():
        bad_ins_samples["zero"].append(j)
    elif ins_d[j]['YES'] < 10:
        bad_ins_samples["less_10"].append(j)
    elif ins_d[j]['YES'] < 15:
        bad_ins_samples["less_15"].append(j)
    else:
        good_ins_samples.append(j)

bad_c_samples = {"zero": [], "less_10": [], "less_15": []}
good_c_samples = []
for j in nins_d:
    if 'YES' not in nins_d[j].keys():
        bad_c_samples["zero"].append(j)
    elif nins_d[j]['YES'] < 10:
        bad_c_samples["less_10"].append(j)
    elif nins_d[j]['YES'] < 15:
        bad_c_samples["less_15"].append(j)
    else:
        good_c_samples.append(j)
print("bad ins samples: ", len(bad_ins_samples["less_15"]))
print("bad nins samples: ", len(bad_c_samples["less_15"]))

In [ ]:
#save the bad samples
with open("/datadrive/pavan/az_storage/data_unorganized/skill_graph/version2/bad_ins_samples.json", "w") as f:
    json.dump(bad_ins_samples, f, indent=4)
with open("/datadrive/pavan/az_storage/data_unorganized/skill_graph/version2/bad_c_samples.json", "w") as f:
    json.dump(bad_c_samples, f, indent=4)

## Add good templates to the graph

In [ ]:
with open("./indicator_graph_edges_weighted.pkl", "rb") as f:
    DG = pickle.load(f)

In [ ]:
for node in DG.nodes():
    if node in good_ins_samples:
        DG.nodes[node]['ins_templates'] = ins_data[node]
    if node in good_c_samples:
        DG.nodes[node]['context_templates'] = nins_data[node]

In [ ]:
with open("./graph_with_good_templates.pkl", "wb") as f:
    pickle.dump(DG, f)

## Correct bad templates

### Seed

In [ ]:
with open("./indicator_graph_edges_weighted.pkl", "rb") as f:
    DG = pickle.load(f)

with open("./bad_ins_samples.json", "r") as f:
    bad_ins = json.load(f)

with open("./bad_c_samples.json", "r") as f:
    bad_c = json.load(f)

In [ ]:
bad_ins = bad_ins['less_15'] + bad_ins['less_10'] + bad_ins['zero']
bad_c = bad_c['less_15'] + bad_c['less_10'] + bad_c['zero']

bad_ins_seed = []
bad_c_seed = []

for node in DG.nodes():
    node_stage = DG.nodes[node].get('stage', None)
    node_skill = DG.nodes[node].get('skill', None)
    node_subskill = DG.nodes[node].get('subskill', None)
    node_goal = DG.nodes[node].get('goal', None)
    node_age_group = DG.nodes[node].get('age_group', None)
    node_indicator = DG.nodes[node].get('label', None)
    node_id = node

    if node in bad_ins:
        bad_ins_seed.append({
            "indicator": node_indicator,
            "age_group": node_age_group,
            "skill": node_skill,
            "subskill": node_subskill,
            "goal": node_goal,
            "stage": node_stage,
            "id" : node_id
        })

    if node in bad_c:
        bad_c_seed.append({
            "indicator": node_indicator,
            "age_group": node_age_group,
            "skill": node_skill,
            "subskill": node_subskill,
            "goal": node_goal,
            "stage": node_stage,
            "id" : node_id
        })
        
with open("./bad_ins_seed.jsonl", "w") as f:
    json.dump(bad_ins_seed, f, indent=4)

with open("./bad_c_seed.jsonl", "w") as f:
    json.dump(bad_c_seed, f, indent=4)
    
with open("./text_type_prompt4.1.json", "r") as f:
    bad_ins_prompt = json.load(f)

with open("./text_type_prompt4.2.json", "r") as f:
    bad_c_prompt = json.load(f)

### Post-processing generations

In [ ]:
with open("./bad_ins_raw.jsonl", "r") as f:
    bad_ins_data = json.load(f)

In [ ]:
bad_ins_all_dicts = [o for o in bad_ins_data if isinstance(o['output'], dict) and isinstance(o['output']['text_types'], list)]
bad_ins_non_dicts = [o for o in bad_ins_data if not isinstance(o['output'], dict) or not isinstance(o['output']['text_types'], list)]
print(len(bad_ins_all_dicts)), print(len(bad_ins_non_dicts))

with open("./bad_c_raw.jsonl", "r") as f:
    bad_c_data = json.load(f)

bad_c_all_dicts = [o for o in bad_c_data if isinstance(o['output'], dict) and isinstance(o['output']['text_types'], list)]
bad_c_non_dicts = [o for o in bad_c_data if not isinstance(o['output'], dict) or not isinstance(o['output']['text_types'], list)]
print(len(bad_c_all_dicts)), print(len(bad_c_non_dicts))

In [ ]:
with open("./indicator_graph_edges_weighted.pkl", "rb") as f:
    DG = pickle.load(f)

ver_bad_ins_seed = []
ver_bad_c_seed = []

for node_data in bad_c_all_dicts:
    node = node_data['id']
    node_stage = DG.nodes[node].get('stage', None)
    node_skill = DG.nodes[node].get('skill', None)
    node_subskill = DG.nodes[node].get('subskill', None)
    node_goal = DG.nodes[node].get('goal', None)
    node_age_group = DG.nodes[node].get('age_group', None)
    node_indicator = DG.nodes[node].get('label', None)
    node_id = node
    tt = node_data['output']['text_types']
    for t in tt:
        ver_bad_c_seed.append({
            "indicator": node_indicator,
            "age_group": node_age_group,
            "skill": node_skill,
            "subskill": node_subskill,
            "goal": node_goal,
            "stage": node_stage,
            "text_type" : t,
            "id" : node_id
        })

for node_data in bad_ins_all_dicts:
    node = node_data['id']
    node_stage = DG.nodes[node].get('stage', None)
    node_skill = DG.nodes[node].get('skill', None)
    node_subskill = DG.nodes[node].get('subskill', None)
    node_goal = DG.nodes[node].get('goal', None)
    node_age_group = DG.nodes[node].get('age_group', None)
    node_indicator = DG.nodes[node].get('label', None)
    node_id = node
    tt = node_data['output']['text_types']
    for t in tt:
        ver_bad_ins_seed.append({
            "indicator": node_indicator,
            "age_group": node_age_group,
            "skill": node_skill,
            "subskill": node_subskill,
            "goal": node_goal,
            "stage": node_stage,
            "text_type" : t,
            "id" : node_id
        })

In [ ]:
with open("./ver_bad_c_seed.jsonl", "w") as f:
    json.dump(ver_bad_c_seed, f, indent=4)

with open("./ver_bad_ins_seed.jsonl", "w") as f:
    json.dump(ver_bad_ins_seed, f, indent=4)

In [ ]:
with open("./ver_bad_ins_raw.jsonl", "r") as f:
    ver_bad_ins_data = json.load(f)

with open("./ver_bad_c_raw.jsonl", "r") as f:
    ver_bad_c_data = json.load(f)

In [ ]:
ver_bad_ins_all_dicts = [o for o in ver_bad_ins_data if isinstance(o['output'], dict) and o['output']['is_valid'] in ["YES", "NO"]]
ver_bad_ins_non_dicts = [o for o in ver_bad_ins_data if not isinstance(o['output'], dict) or o['output']['is_valid'] not in ["YES", "NO"]]
print(len(ver_bad_ins_all_dicts)), print(len(ver_bad_ins_non_dicts))
ver_bad_c_all_dicts = [o for o in ver_bad_c_data if isinstance(o['output'], dict) and o['output']['is_valid'] in ["YES", "NO"]]
ver_bad_c_non_dicts = [o for o in ver_bad_c_data if not isinstance(o['output'], dict) or o['output']['is_valid'] not in ["YES", "NO"]]
print(len(ver_bad_c_all_dicts)), print(len(ver_bad_c_non_dicts))

In [ ]:
ver_bad_ins_results = {}
ver_bad_c_results = {}
for i in ver_bad_c_all_dicts:
    ver_bad_c_results[i['id']] = ver_bad_c_results.get(i['id'], {})
    ver_bad_c_results[i['id']][i['output']['is_valid']] = ver_bad_c_results[i['id']].get(i['output']['is_valid'], [])
    ver_bad_c_results[i['id']][i['output']['is_valid']].append(i['text_type'])
for i in ver_bad_ins_all_dicts:
    ver_bad_ins_results[i['id']] = ver_bad_ins_results.get(i['id'], {})
    ver_bad_ins_results[i['id']][i['output']['is_valid']] = ver_bad_ins_results[i['id']].get(i['output']['is_valid'], [])
    ver_bad_ins_results[i['id']][i['output']['is_valid']].append(i['text_type'])

In [ ]:
for i in ver_bad_c_results:
    if 'YES' in ver_bad_c_results[i].keys():
        if len(ver_bad_c_results[i]['YES']) < 15:
            #Sample from No text types until 15 length
            while len(ver_bad_c_results[i]['YES']) < 15:
                if len(ver_bad_c_results[i]['NO']) == 0:
                    break
                ver_bad_c_results[i]['YES'].append(ver_bad_c_results[i]['NO'].pop(0))
    else:
        ver_bad_c_results[i]['YES'] = []
        while len(ver_bad_c_results[i]['YES']) < 15:
            if len(ver_bad_c_results[i]['NO']) == 0:
                break
            ver_bad_c_results[i]['YES'].append(ver_bad_c_results[i]['NO'].pop(0))

for i in ver_bad_ins_results:
    if 'YES' in ver_bad_ins_results[i].keys():
        if len(ver_bad_ins_results[i]['YES']) < 15:
            #Sample from No text types until 15 length
            while len(ver_bad_ins_results[i]['YES']) < 15:
                if len(ver_bad_ins_results[i]['NO']) == 0:
                    break
                ver_bad_ins_results[i]['YES'].append(ver_bad_ins_results[i]['NO'].pop(0))
    else:
        ver_bad_ins_results[i]['YES'] = []
        while len(ver_bad_ins_results[i]['YES']) < 15:
            if len(ver_bad_ins_results[i]['NO']) == 0:
                break
            ver_bad_ins_results[i]['YES'].append(ver_bad_ins_results[i]['NO'].pop(0))

### Save the graph with templates

In [ ]:
with open("./graph_with_good_templates.pkl", "rb") as f:
    DG = pickle.load(f)

In [ ]:
for i in ver_bad_ins_results:
    DG.nodes[i]['ins_templates'] = ver_bad_ins_results[i]['YES']

for i in ver_bad_c_results:
    DG.nodes[i]['context_templates'] = ver_bad_c_results[i]['YES']


In [ ]:
with open("./graph_final.pkl", "wb") as f:
    pickle.dump(DG, f)